In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import tensorflow as tf
import numpy as np
import keras

import sys
sys.path.append("scripts")
import utils
from PET import PET

import matplotlib.pyplot as plt

In [ ]:
EPOCHS = 5
BATCH_SIZE = 512
LR = 1e-5

NTRAIN = 100000
NVAL = 20000

In [ ]:
tf.config.run_functions_eagerly(False)

In [ ]:
import awkward
import vector

def to_p4(p4_obj):
    return vector.awk(
        awkward.zip(
            {
                "mass": p4_obj.tau,
                "x": p4_obj.x,
                "y": p4_obj.y,
                "z": p4_obj.z,
            }
        )
    )

def deltaphi(phi1, phi2):
    diff = phi1 - phi2
    return np.arctan2(np.sin(diff), np.cos(diff))

def deltar(eta1, phi1, eta2, phi2):
    deta = eta1 - eta2
    dphi = deltaphi(phi1, phi2)
    return np.sqrt(deta**2 + dphi**2)

#initialize weights of model1 from weights of model2.
#ignore layers that don't match in shape.
def set_matching_weights(model1, model2):
    il1 = len(model1.layers)
    il2 = len(model2.layers)
    maxl = max(il1, il1)
    for il in range(maxl):
        if il<il1 and il<il2:
            if len(model1.layers[il].weights)>0:
                if len(model1.layers[il].weights) == len(model2.layers[il].weights):
                    weights_match = True
                    for w1, w2 in zip(model1.layers[il].weights, model2.layers[il].weights):
                        if w1.shape != w2.shape:
                            weights_match = False
                    if weights_match:
                        print(model1.layers[il].name, model2.layers[il].name, [w.shape.as_list() for w in model1.layers[il].weights])
                        model1.layers[il].set_weights(model2.layers[il].weights)

Define the backbone model

In [ ]:
#Define original model to get the weights
model = PET(
    num_feat=13,
    num_jet=4,
    num_classes=10, #this is the number of target classes the original model was trained with. not relevant for us.
    local=True,
    num_layers=8,
    drop_probability=0,
    simple=False,
    layer_scale=True,
    talking_head=False,
    mode="all"
)

Run the backbone model on dummy data to initialize weights.

In [ ]:
x = {}
x["input_features"] = tf.zeros((32, 100, 13))
x["input_points"] = tf.zeros((32, 100, 2))
x["input_mask"] = tf.zeros((32, 100))
x["input_jet"] = tf.zeros((32, 4)) #how many features per jet
x["input_time"] = tf.zeros((32, 1)) #this is not used

model(x)
out = model.body(x)
print(len(out), out.shape)

Load the weights of the backbone model.

In [ ]:
model.summary()
model.load_weights("checkpoints/PET_jetclass_8_local_layer_scale_token_baseline_all.weights.h5", by_name=True)

In [ ]:
data = awkward.from_parquet("zh.parquet")

In [ ]:
reco_cand_p4s = to_p4(data["reco_cand_p4s"])
reco_jet_p4s = to_p4(data["reco_jet_p4s"])

delta_eta = reco_cand_p4s.eta - reco_jet_p4s.eta
delta_phi = deltaphi(reco_cand_p4s.phi, reco_jet_p4s.phi)
log_pt = np.log(reco_cand_p4s.pt)
log_e = np.log(reco_cand_p4s.energy)
log_ptjet = np.log(1 - reco_cand_p4s.pt/reco_jet_p4s.pt)
log_ejet = np.log(1 - reco_cand_p4s.energy/reco_jet_p4s.energy)
delta_r = deltar(reco_cand_p4s.eta, reco_cand_p4s.phi, reco_jet_p4s.eta, reco_jet_p4s.phi)
charge = data["reco_cand_charge"]
is_ele = np.abs(data["reco_cand_pdg"])==11
is_mu = np.abs(data["reco_cand_pdg"])==13
is_photon = np.abs(data["reco_cand_pdg"])==22
is_chhad = np.abs(data["reco_cand_pdg"])==210
is_nhad = np.abs(data["reco_cand_pdg"])==130

In [ ]:
pad_size = 32 #max number of particles per jet
fill_val = 0 #fill value of padded data

#create particle array in the shape [njets, pad_size, 13]
vals = [
    awkward.to_numpy(
        awkward.fill_none(
            awkward.pad_none(
                x, pad_size, clip=True), fill_val
        )
    ) for x in [delta_eta, delta_phi, log_pt, log_e, log_ptjet, log_ejet, delta_r, charge, is_ele, is_mu, is_photon, is_chhad, is_nhad]
]
particles = np.stack(vals, axis=-1)
particles[np.isnan(particles)] = 0
particles[np.isinf(particles)] = 0
particles_mask = (~awkward.to_numpy(awkward.pad_none(delta_eta, pad_size, clip=True)).mask).astype(np.float32)

#normalize particles
means_particle = particles[np.squeeze(particles_mask==1)].mean(axis=0)
stds_particle = particles[np.squeeze(particles_mask==1)].std(axis=0)
stds_particle[stds_particle==0] = 1
particles = (particles - means_particle)/stds_particle

#create jet array in the shape [njets, 4]
jets = awkward.to_numpy(np.stack([
    reco_jet_p4s.pt,
    reco_jet_p4s.eta,
    reco_jet_p4s.mass,
    awkward.num(reco_cand_p4s)], axis=-1)
)
jets[np.isnan(jets)] = 0
jets[np.isinf(jets)] = 0

#normalize jets
means_jet = jets.mean(axis=0)
stds_jet = jets.std(axis=0)
stds_jet[stds_jet==0] = 1
jets = (jets - means_jet)/stds_jet


In [ ]:
means_particle, stds_particle

In [ ]:
means_jet, stds_jet

In [ ]:
particles.shape, particles_mask.shape, jets.shape

In [ ]:
targets = awkward.to_numpy(data["gen_jet_tau_decaymode"])

In [ ]:
def prepare_data(start, stop):
    x = {}
    x["input_features"] = particles[start:stop]
    x["input_points"] = particles[start:stop, :, :2]
    x["input_mask"] = np.expand_dims(particles_mask[start:stop], axis=-1)
    x["input_jet"] = jets[start:stop]
    x["input_time"] = np.zeros((stop-start, 1))
    y = targets[start:stop]
    return x, y

In [ ]:
from tensorflow.keras import layers
from layers import StochasticDepth, TalkingHeadAttention, LayerScale, RandomDrop

def get_encoding(x, projection_dim, use_bias=True):
    x = layers.Dense(2*projection_dim, use_bias=use_bias, activation='gelu')(x)
    x = layers.Dense(projection_dim, use_bias=use_bias, activation='gelu')(x)
    return x

def FourierProjection(x,projection_dim,num_embed=64):    
    half_dim = num_embed // 2
    emb = tf.math.log(10000.0) / (half_dim - 1)
    emb = tf.cast(emb,tf.float32)
    freq = tf.exp(-emb* tf.range(start=0, limit=half_dim, dtype=tf.float32))


    angle = x*freq*1000.0
    embedding = tf.concat([tf.math.sin(angle),tf.math.cos(angle)],-1)*x
    embedding = layers.Dense(2*projection_dim,activation="swish",use_bias=False)(embedding)
    embedding = layers.Dense(projection_dim,activation="swish",use_bias=False)(embedding)
    
    return embedding

def knn(num_points, k, topk_indices, features):
    # topk_indices: (N, P, K)
    # features: (N, P, C)    
    batch_size = tf.shape(features)[0]

    batch_indices = tf.reshape(tf.range(batch_size), (-1, 1, 1))
    batch_indices = tf.tile(batch_indices, (1, num_points, k))
    indices = tf.stack([batch_indices, topk_indices], axis=-1)
    return tf.gather_nd(features, indices)

def get_neighbors(points,features,projection_dim,K):
    drij = pairwise_distance(points)  # (N, P, P)
    _, indices = tf.nn.top_k(-drij, k=K + 1)  # (N, P, K+1)
    indices = indices[:, :, 1:]  # (N, P, K)
    knn_fts = knn(tf.shape(points)[1], K, indices, features)  # (N, P, K, C)
    knn_fts_center = tf.broadcast_to(tf.expand_dims(features, 2), tf.shape(knn_fts))
    local = tf.concat([knn_fts-knn_fts_center,knn_fts_center],-1)
    local = layers.Dense(2*projection_dim,activation='gelu')(local)
    local = layers.Dense(projection_dim,activation='gelu')(local)
    local = tf.reduce_mean(local,-2)
    
    return local

def pairwise_distance(point_cloud):
    r = tf.reduce_sum(point_cloud * point_cloud, axis=2, keepdims=True)
    m = tf.matmul(point_cloud, point_cloud, transpose_b = True)
    D = r - 2 * m + tf.transpose(r, perm=(0, 2, 1)) + 1e-5
    return D


class TransformerModel(keras.Model):
    def __init__(self,
                 use_backbone,
                 num_feat,
                 num_jet,      
                 num_classes=2):
        
        super(TransformerModel, self).__init__()
        
        self.projection_dim = model.projection_dim
        self.num_heads = model.num_heads
        self.num_classes = num_classes
        self.class_activation = "softmax"
        
        self.feature_drop = model.feature_drop
        self.num_keep = model.num_keep
        self.mode = model.mode
        self.num_layers = model.num_layers
        self.layer_scale = model.layer_scale
        self.layer_scale_init = model.layer_scale_init
        self.drop_probability = model.drop_probability
        self.dropout = model.dropout
        self.num_class_layers = 2
        
        self._input_features = layers.Input(shape=(None, num_feat), name='input_features')
        self._input_points = layers.Input(shape=(None, 2), name='input_points')
        self._input_mask = layers.Input(shape=(None, 1), name='input_mask')
        self._input_jet = layers.Input((num_jet, ),name='input_jet')
        self._input_time = layers.Input((None, ),name='input_time')

        if use_backbone:
            self.backbone_body = self.PET_body(
                self._input_features,
                self._input_points,
                self._input_mask,
                self._input_time,
                True,
                self.num_classes,
                2,
                False
            )
            self.backbone = keras.Model(
                inputs=[self._input_features, self._input_points, self._input_mask, self._input_time],
                outputs=[self.backbone_body], name="backbone"
            )

            particles_encoded = self.backbone_body
        else:
            particles_encoded = get_encoding(self._input_features, self.projection_dim)

        classifier_out, regression_out = self.PET_classifier(
            particles_encoded, self._input_jet, self.num_class_layers, num_jet
        )
        self.classifier = keras.Model(
            inputs=[self._input_features, self._input_points, self._input_mask, self._input_jet, self._input_time],
            outputs=[classifier_out], name="classifier"
        )
            
    def PET_classifier(
            self,
            encoded,
            input_jet,
            num_class_layers,
            num_jet,
            simple = False
    ):

        #Include event information as a representative particle
        if simple:
            encoded = layers.GroupNormalization(groups=1)(encoded)
            representation = layers.GlobalAveragePooling1D()(encoded)
            jet_encoded = get_encoding(input_jet,self.projection_dim)
            representation = layers.Dense(self.projection_dim,activation='gelu')(representation+jet_encoded)
            outputs_pred = layers.Dense(self.num_classes,activation=self.class_activation)(representation)
            outputs_mse = layers.Dense(num_jet)(representation)
        else:
            conditional = layers.Dense(2*self.projection_dim,activation='gelu')(input_jet)
            conditional = tf.tile(conditional[:,None, :], [1,tf.shape(encoded)[1], 1])
            scale,shift = tf.split(conditional,2,-1)
            encoded = encoded*(1.0 + scale) + shift

            class_tokens = tf.Variable(tf.zeros(shape=(1, self.projection_dim)),trainable = True)    
            class_tokens = tf.tile(class_tokens[None, :, :], [tf.shape(encoded)[0], 1, 1])
                        
            for _ in range(num_class_layers):
                concatenated = tf.concat([class_tokens, encoded],1)

                x1 = layers.GroupNormalization(groups=1)(concatenated)            
                updates = layers.MultiHeadAttention(num_heads=self.num_heads,
                                                    key_dim=self.projection_dim//self.num_heads)(
                                                        query=x1[:,:1], value=x1, key=x1)
                updates = layers.GroupNormalization(groups=1)(updates)
                if self.layer_scale:
                    updates = LayerScale(self.layer_scale_init, self.projection_dim)(updates)

                x2 = layers.Add()([updates,class_tokens])
                x3 = layers.GroupNormalization(groups=1)(x2)
                x3 = layers.Dense(2*self.projection_dim,activation="gelu")(x3)
                x3 = layers.Dropout(self.dropout)(x3)
                x3 = layers.Dense(self.projection_dim)(x3)
                if self.layer_scale:
                    x3 = LayerScale(self.layer_scale_init, self.projection_dim)(x3)
                class_tokens = layers.Add()([x3,x2])


            class_tokens = layers.GroupNormalization(groups=1)(class_tokens)
            outputs_pred = layers.Dense(self.num_classes,activation=self.class_activation)(class_tokens[:,0])
            outputs_mse = layers.Dense(num_jet)(class_tokens[:,0])

        return outputs_pred,outputs_mse
    
    def PET_body(self,
                 input_features,
                 input_points,
                 input_mask,
                 input_time,
                 local, K,num_local,
                 talking_head,
                 ):
            
        #Randomly drop features not present in other datasets
        encoded = RandomDrop(self.feature_drop if  'all' in self.mode else 0.0,num_skip=self.num_keep)(input_features)                        
        encoded = get_encoding(encoded,self.projection_dim)

        time = FourierProjection(input_time,self.projection_dim)
        time = tf.tile(time[:,None, :], [1,tf.shape(encoded)[1], 1])*input_mask
        time = layers.Dense(2*self.projection_dim,activation='gelu',use_bias=False)(time)
        scale,shift = tf.split(time,2,-1)
        
        encoded = encoded*(1.0+scale) + shift
        
        if local:
            coord_shift = tf.multiply(999., tf.cast(tf.equal(input_mask, 0), dtype='float32'))        
            points = input_points[:,:,:2]
            local_features = input_features
            for _ in range(num_local):
                local_features = get_neighbors(coord_shift+points,local_features,self.projection_dim,K)
                points = local_features
                
            encoded = layers.Add()([local_features,encoded])

        skip_connection = encoded
        for i in range(self.num_layers):
            x1 = layers.GroupNormalization(groups=1)(encoded)
            if talking_head:
                updates, _ = TalkingHeadAttention(self.projection_dim, self.num_heads, 0.0)(x1)
            else:
                updates = layers.MultiHeadAttention(num_heads=self.num_heads,
                                                    key_dim=self.projection_dim//self.num_heads)(x1,x1)

            if self.layer_scale:
                updates = LayerScale(self.layer_scale_init, self.projection_dim)(updates,input_mask)
            updates = StochasticDepth(self.drop_probability)(updates)
            x2 = layers.Add()([updates,encoded])
            x3 = layers.GroupNormalization(groups=1)(x2)
            x3 = layers.Dense(2*self.projection_dim,activation="gelu")(x3)
            x3 = layers.Dropout(self.dropout)(x3)
            x3 = layers.Dense(self.projection_dim)(x3)
            if self.layer_scale:
                x3 = LayerScale(self.layer_scale_init, self.projection_dim)(x3,input_mask)
            x3 = StochasticDepth(self.drop_probability)(x3)
            encoded = layers.Add()([x3,x2])*input_mask
        return encoded + skip_connection
    
    def call(self, x):
        ret = self.classifier([
            x["input_features"], x["input_points"], x["input_mask"], x["input_jet"], x["input_time"]
        ])
        return ret

In [ ]:
#model without backbone
model_dm_direct = TransformerModel(False, 13, 4, 16)
x, y = prepare_data(0, 256)
model_dm_direct(x)

#model with backbone, initialization from scratch
model_dm_bb = TransformerModel(True, 13, 4, 16)
x, y = prepare_data(0, 256)
model_dm_bb(x)

In [ ]:
model_dm_direct.summary()

In [ ]:
model_dm_bb.summary()

In [ ]:
X_train, y_train = prepare_data(0, NTRAIN)
X_val, y_val = prepare_data(NTRAIN, NTRAIN+NVAL)

In [ ]:
plt.hist(np.sum(X_train["input_mask"], axis=1), bins=np.linspace(0,32,33), density=1, histtype="step", lw=2, label="train");
plt.hist(np.sum(X_val["input_mask"], axis=1), bins=np.linspace(0,32,33), density=1, histtype="step", lw=2, label="val");
plt.xlabel("number of particles per jet")
plt.legend()

In [ ]:
b = np.linspace(-10,10,100)

X_train_flat = X_train["input_features"][np.squeeze(X_train["input_mask"]==1)]
X_val_flat = X_val["input_features"][np.squeeze(X_val["input_mask"]==1)]

In [ ]:
plt.hist(
    X_train_flat[:, 0],
    bins=b, density=1, histtype="step", lw=2, label="train"
);

plt.hist(
    X_val_flat[:, 0],
    bins=b, density=1, histtype="step", lw=2, label="val"
);
plt.yscale("log");
plt.xlabel("feature 0: delta_eta")
plt.legend()

In [ ]:
plt.hist(
    X_train_flat[:, 1],
    bins=b, density=1, histtype="step", lw=2, label="train"
);

plt.hist(
    X_val_flat[:, 1],
    bins=b, density=1, histtype="step", lw=2, label="val"
);
plt.yscale("log");
plt.xlabel("feature 0: delta_phi")
plt.legend()

In [ ]:
model_dm_direct.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LR),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False)
)
history = model_dm_direct.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val)
)

In [ ]:
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title("Direct model")
plt.legend()

In [ ]:
encodings_init = model_dm_bb.backbone(x)

model_dm_bb.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LR),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False)
)
history2 = model_dm_bb.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val)
)

encodings_init_trained = model_dm_bb.backbone(x)

In [ ]:
plt.plot(history2.history["loss"], label="train")
plt.plot(history2.history["val_loss"], label="val")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title("OmniJet naive model")
plt.legend()

In [ ]:
#model with backbone, initialize from checkpoint
model_dm_bb_cp = TransformerModel(True, 13, 4, 16)
x, y = prepare_data(0, 256)
model_dm_bb_cp(x)

In [ ]:
#set backbone weights
outs_pre = model_dm_bb_cp.backbone(x)
plt.scatter(
    outs_pre[tf.squeeze(x["input_mask"]==1)][:, 0],
    outs_pre[tf.squeeze(x["input_mask"]==1)][:, 1],
    marker="."
);

set_matching_weights(model_dm_bb_cp.backbone, model.body)

outs_post = model_dm_bb_cp.backbone(x)
plt.scatter(
    outs_post[tf.squeeze(x["input_mask"]==1)][:, 0],
    outs_post[tf.squeeze(x["input_mask"]==1)][:, 1],
    marker="."
);

In [ ]:
outs_pre = model_dm_bb_cp(x)
plt.scatter(
    outs_pre[:, 0],
    outs_pre[:, 1],
    marker="."
);
set_matching_weights(model_dm_bb_cp.classifier, model.classifier)

outs_post = model_dm_bb_cp(x)
plt.scatter(
    outs_post[:, 0],
    outs_post[:, 1],
    marker="."
);

In [ ]:
encodings_before_finetune = model_dm_bb_cp.backbone(x)

model_dm_bb_cp.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LR),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False)
)
history3 = model_dm_bb_cp.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val)
)

encodings_after_finetune = model_dm_bb_cp.backbone(x)

In [ ]:
b = np.linspace(-10,10,100)
plt.hist(
    encodings_init[tf.squeeze(x["input_mask"]==1)][:, 0],
    bins=b,
    label="OmniLearn init", histtype="step", lw=2
);

plt.hist(
    encodings_init_trained[tf.squeeze(x["input_mask"]==1)][:, 0],
    bins=b,
    label="OmniLearn init + train", histtype="step", lw=2
);

plt.hist(
    encodings_before_finetune[tf.squeeze(x["input_mask"]==1)][:, 0],
    bins=b,
    label="OmniLearn checkpoint", histtype="step", lw=2
);

plt.hist(
    encodings_after_finetune[tf.squeeze(x["input_mask"]==1)][:, 0],
    bins=b,
    label="OmniLearn checkpoint + train", histtype="step", lw=2
);

plt.legend(loc="best")
plt.xlabel("encoded feature 0")

In [ ]:
plt.scatter(
    encodings_init[tf.squeeze(x["input_mask"]==1)][:, 0],
    encodings_init[tf.squeeze(x["input_mask"]==1)][:, 1],
    marker=".",
    alpha=0.5,
    label="OmniLearn init"
);

plt.scatter(
    encodings_init_trained[tf.squeeze(x["input_mask"]==1)][:, 0],
    encodings_init_trained[tf.squeeze(x["input_mask"]==1)][:, 1],
    marker=".",
    alpha=0.5,
    label="OmniLearn init + train"
);

plt.scatter(
    encodings_before_finetune[tf.squeeze(x["input_mask"]==1)][:, 0],
    encodings_before_finetune[tf.squeeze(x["input_mask"]==1)][:, 1],
    marker=".",
    alpha=0.5,
    label="OmniLearn checkpoint"
);

plt.scatter(
    encodings_after_finetune[tf.squeeze(x["input_mask"]==1)][:, 0],
    encodings_after_finetune[tf.squeeze(x["input_mask"]==1)][:, 1],
    marker=".",
    alpha=0.5,
    label="OmniLearn checkpoint + train"
);

plt.legend(loc="best")
plt.xlabel("encoded feature 0")
plt.ylabel("encoded feature 1")

In [ ]:
plt.plot(history3.history["loss"], label="train")
plt.plot(history3.history["val_loss"], label="val")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title("OmniJet checkpoint model")
plt.legend()

In [ ]:
plt.plot(history.history["loss"], label="direct", color="black")
plt.plot(history2.history["loss"], label="OmniLearn naive + train", color="red")
plt.plot(history3.history["loss"], label="OmniLearn checkpoint + train", color="orange")
plt.legend()
plt.ylabel("train loss")

In [ ]:
plt.plot(history.history["val_loss"], label="direct", color="black")
plt.plot(history2.history["val_loss"], label="OmniLearn naive + train", color="red")
plt.plot(history3.history["val_loss"], label="OmniLearn checkpoint + train", color="orange")
plt.legend()
plt.ylabel("validation loss")